Download dataset from https://phonbank.talkbank.org/phon/

In [1]:
# https://www.nltk.org/_modules/nltk/corpus/reader/childes.html

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from childes_mi.utils.paths import DATA_DIR, ensure_dir, PHONBANK_DIR

In [4]:
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import requests
import urllib.request
from tqdm.autonotebook import tqdm
import zipfile

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### get URLs to download

In [5]:
def isDirectory(url):
    if(url.endswith('/')):
        return True
    else:
        return False

def findLinks(url):
    page = requests.get(url).content
    bsObj = BeautifulSoup(page, 'html.parser')
    maybe_directories = bsObj.findAll('a', href=True)
    links = []
    for link in maybe_directories:
        #print(link)
        if(isDirectory(link['href'])):
            if link['href'] == '/': continue
            newUrl = url + link['href']  
            #print('newurl', newUrl)
            links.append(findLinks(newUrl)) #recursion happening here
        else:
            if(link['href'].endswith('.zip')):
                print("GOTCHA!", url + link['href']) #now safe and download
                links.append(url + link['href'])
    return links

In [6]:
startUrl = "https://phonbank.talkbank.org/phon/"
urls = findLinks(startUrl)

GOTCHA! https://phonbank.talkbank.org/phon/Biling/Almeida.zip
GOTCHA! https://phonbank.talkbank.org/phon/Biling/FallsChurch.zip
GOTCHA! https://phonbank.talkbank.org/phon/Biling/PraisleyRose.zip
GOTCHA! https://phonbank.talkbank.org/phon/Biling/Scheidnes.zip
GOTCHA! https://phonbank.talkbank.org/phon/Biling/VYSA.zip
GOTCHA! https://phonbank.talkbank.org/phon/Chinese/Cantonese/PaidoCantonese.zip
GOTCHA! https://phonbank.talkbank.org/phon/Chinese/Taiwanese/Tsay.zip
GOTCHA! https://phonbank.talkbank.org/phon/Clinical/Bernhardt.zip
GOTCHA! https://phonbank.talkbank.org/phon/Clinical/Chiat.zip
GOTCHA! https://phonbank.talkbank.org/phon/Clinical/Cummings.zip
GOTCHA! https://phonbank.talkbank.org/phon/Clinical/Granada.zip
GOTCHA! https://phonbank.talkbank.org/phon/Clinical/McAllisterByun.zip
GOTCHA! https://phonbank.talkbank.org/phon/Clinical/Preston.zip
GOTCHA! https://phonbank.talkbank.org/phon/Clinical/TorringtonEaton.zip
GOTCHA! https://phonbank.talkbank.org/phon/Dutch/CLPF.zip
GOTCHA! ht

In [7]:
# flatten the recursive directiories
flatten = lambda l: sum(map(flatten, l), []) if isinstance(l, list) else [l]
urls = flatten(urls)
urls[:3]

['https://phonbank.talkbank.org/phon/Biling/Almeida.zip',
 'https://phonbank.talkbank.org/phon/Biling/FallsChurch.zip',
 'https://phonbank.talkbank.org/phon/Biling/PraisleyRose.zip']

### download from URLs

In [10]:
output_directory = PHONBANK_DIR
for url in tqdm(urls):
    # get save location
    save_loc = output_directory / url[35:]
    # make sure there is a directory to save to
    ensure_dir(save_loc)
    # save to disk
    urllib.request.urlretrieve(url, save_loc)

### unzip files

In [11]:
zip_files = list(PHONBANK_DIR.glob('**/*.zip'))
zip_files[:3]

[PosixPath('/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/PHON_Data_Dir/corpora/Spanish/Koine.zip'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/PHON_Data_Dir/corpora/Spanish/LlinasOjea.zip'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/PHON_Data_Dir/corpora/Spanish/Vivar.zip')]

In [12]:
for zf in tqdm(zip_files):
    with zipfile.ZipFile(zf, 'r') as zip_ref:
        zip_ref.extractall(zf.parent)